<a href="https://colab.research.google.com/github/shirazgha/ExamTP_AG/blob/main/TSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The Traveling Salesperson Problem**

Le problème du voyageur de commerce (TSP) pose la question suivante : "Étant donné une liste de villes et les distances entre chaque paire de villes, quel est l'itinéraire le plus court possible qui visite chaque ville et retourne à la ville d'origine ?

In [14]:
import random
import math

POP_SIZE = 15 #nbre de chromosomes
NB_GENES = 15 
MUT_RATE = 0.2 #taux de mutation
CROSS_RATE = 0.7 #taux de croisement

TOURNAMENT_SELECTION_SIZE = 4

# nombre de combinaisons possible = factoriel de NB_GENES 
# donc pour eviter la redandance des individus(villes) on peut definir la taille de la population inferieur ou egale aux total des combinaisons

Villes = []
i=0
while i < NB_GENES:
  ville = (random.randint(0,9),random.randint(0,9))
  if ville not in Villes:
    Villes.append(ville)
    i += 1

In [15]:
Villes

[(3, 4),
 (5, 8),
 (0, 6),
 (5, 1),
 (7, 7),
 (0, 4),
 (1, 7),
 (0, 3),
 (6, 9),
 (5, 0),
 (9, 8),
 (7, 2),
 (8, 2),
 (6, 3),
 (0, 2)]

In [16]:
class Chromosome:
    '''Chromosome Init'''
    def __init__(self, villes):
        self.genes = []
        self.fitness = 0
        i = 0
        while i < NB_GENES:
          ville = Villes[random.randint(0,NB_GENES - 1)]
          if ville not in self.genes:
            self.genes.append(ville)
            i += 1
         
          
    def get_genes(self):
        return self.genes

    def get_X(self,pos):
        return self.genes[pos][0]

    def get_Y(self, pos):
        return self.genes[pos][1]

    def get_fitness(self):
        self.fitness = 0

        for i in range(NB_GENES-1):
          #distance = math.sqrt((x1 - x2)**2 + (y1 - y2)**2)
                # self.fitness += math.sqrt((self.genes[i][0] - self.genes[i+1][0])**2 + (self.genes[i][1] - self.genes[i+1][1])**2)

                self.fitness += math.sqrt((self.get_X(i) - self.get_X(i+1))**2 + (self.get_Y(i) - self.get_Y(i+1))**2)
        self.fitness += math.sqrt((self.get_X(NB_GENES-1) - self.get_X(0))**2 + (self.get_Y(NB_GENES-1) - self.get_Y(0))**2)       
        return round(self.fitness, 2) 

    def __str__(self):
        return self.genes.__str__()

In [4]:
Chromosome(Villes).__str__()

'[(3, 3), (8, 0), (6, 6), (8, 2), (0, 7), (2, 5), (4, 7), (4, 6), (1, 2), (9, 6), (2, 2), (1, 0), (1, 6), (2, 4), (5, 6)]'

In [17]:
class Population:
    '''Population Init'''
    def __init__(self, size):
        self.chromosomes = []
        i = 0
        while i < size :
            self.chromosomes.append( Chromosome(Villes) )
            i += 1
        self.chromosomes.sort(key=lambda x: x.get_fitness(), reverse=False)

    '''Get All Population Chromosomes'''
    def get_chromosomes(self):
        return self.chromosomes

    def print_population(self, gen_number):
        print("\n-----------------------Generation Summary---------------------------")
        print("--------------------------------------------------------------------")
        print("Generation #", gen_number, "| Fittest chromosome fitness:", self.get_chromosomes()[0].get_fitness())
        # print("Target chromosome", TARGET_CHROMOSOME)
        print("--------------------------------------------------------------------")
        i = 0
        for x in self.get_chromosomes():
            print("Chromosome #",i," :",x,"| Fitness", x.get_fitness())
            i += 1

In [6]:
Population(15).print_population(0)


-----------------------Generation Summary---------------------------
--------------------------------------------------------------------
Generation # 0 | Fittest chromosome fitness: 51.41
--------------------------------------------------------------------
Chromosome # 0  : [(4, 6), (2, 5), (1, 6), (5, 6), (0, 7), (4, 7), (3, 3), (2, 2), (1, 0), (1, 2), (2, 4), (8, 2), (8, 0), (6, 6), (9, 6)] | Fitness 51.41
Chromosome # 1  : [(6, 6), (4, 7), (3, 3), (4, 6), (9, 6), (2, 2), (1, 2), (2, 4), (8, 2), (8, 0), (1, 0), (0, 7), (5, 6), (2, 5), (1, 6)] | Fitness 62.89
Chromosome # 2  : [(4, 7), (1, 2), (8, 0), (8, 2), (2, 4), (9, 6), (2, 5), (1, 6), (4, 6), (5, 6), (1, 0), (2, 2), (3, 3), (0, 7), (6, 6)] | Fitness 65.38
Chromosome # 3  : [(1, 6), (2, 4), (0, 7), (1, 2), (2, 2), (4, 6), (6, 6), (1, 0), (3, 3), (8, 2), (5, 6), (4, 7), (2, 5), (8, 0), (9, 6)] | Fitness 66.06
Chromosome # 4  : [(0, 7), (4, 7), (2, 4), (3, 3), (1, 6), (8, 2), (1, 2), (2, 2), (2, 5), (4, 6), (6, 6), (5, 6), (8, 0)

In [18]:
def select_tournament(pop):
    tournament_pop = Population(0)
    i = 0
    while i < TOURNAMENT_SELECTION_SIZE :
        tournament_pop.get_chromosomes().append(pop.get_chromosomes()[random.randrange(0,POP_SIZE)])
        i += 1
    tournament_pop.get_chromosomes().sort(key=lambda x: x.get_fitness(), reverse=False)
    return tournament_pop.get_chromosomes()[0]

In [19]:
def crossover_chromosomes(parent1, parent2):
    if random.random() < CROSS_RATE:
      villes = []
      child1 = Chromosome(villes)
      child2 = Chromosome(villes)

      '''One Point Cross Over'''
      index = random.randrange(1, NB_GENES)
      # child1.genes = parent1.get_genes()[:index] + parent2.get_genes()[index:]
      sub_1_child1 = parent1.get_genes()[:index]
      sub_2_child1 = [item for item in parent2.get_genes() if item not in sub_1_child1]
      child1.genes = sub_1_child1 + sub_2_child1
      # child2.genes = parent2.get_genes()[:index] + parent1.get_genes()[index:]
      sub_1_child2 = parent2.get_genes()[:index]
      sub_2_child2 = [item for item in parent1.get_genes() if item not in sub_1_child2]
      child2.genes = sub_1_child2 + sub_2_child2


      print("\nMaking a cross")
      print("Parent1: ",parent1.get_genes())
      print("Parent2: ",parent2.get_genes())
      print("Child1 : ", child1.get_genes())
      print("Child2 : ", child2.get_genes())

      return child1, child2

    else:
      return parent1, parent2

In [10]:
# cross over with two points 



# def crossover_chromosomes_twoPoints(parent1, parent2):
#     if random.random() < CROSSING_RATE: 
#         child1 = Chromosome(Villes)
#         child2 = Chromosome(Villes)

#         '''One Point Cross Over'''
#         index1 = random.randrange(1, NB_GENES)
#         index2 = random.randrange(1, NB_GENES)
#         while (index2 == index1):
#           index2 = random.randrange(1, NB_GENES)
#         # child1.genes = parent1.get_genes()[:index] + parent2.get_genes()[index:]
#         child1.genes = parent1.get_genes()[:index1] + parent2[index1:index2] + [item for item in parent1.get_genes() if item not in child1.genes]
#         # child2.genes = parent2.get_genes()[:index] + parent1.get_genes()[index:]
#         child2.genes = parent2.get_genes()[:index1] + parent1[index1:index2] + [item for item in parent2.get_genes() if item not in child2.genes]


#         print("\nMaking a cross")
#         print("Parent1: ",parent1.get_genes())
#         print("Parent2: ",parent2.get_genes())
#         print("Child1 : ", child1.get_genes())
#         print("Child2 : ", child2.get_genes())

#         return child1, child2
#     else:
#         return parent1, parent2

In [20]:
def mutatation(individual):
    mutatation_index1 = random.randrange(len(individual.get_genes()))
    mutatation_index2 = random.randrange(len(individual.get_genes()))

    
    individual.get_genes()[mutatation_index1], individual.get_genes()[mutatation_index2] = individual.get_genes()[mutatation_index2], individual.get_genes()[mutatation_index1]


In [21]:
def evolve(pop):
    new_pop = Population(0)
    #'''Keep The Fittests Chromosomes'''
    #for i in range(NUMBER_OF_ELITE_CHROMOSOMES):
    #    new_pop.get_chromosomes().append(pop.get_chromosomes()[i])

    print("\nCrossover and Mutation Trace:")
    while new_pop.get_chromosomes().__len__() < POP_SIZE:
        parent1 = select_tournament(pop)
        parent2 = select_tournament(pop)


        child1, child2 = crossover_chromosomes(parent1, parent2)


        mutatation(child1)
        mutatation(child2)


        new_pop.get_chromosomes().append(child1)

        # make sure to not depass the population size if we keep the elite
        if len(new_pop.get_chromosomes()) < POP_SIZE:
            new_pop.get_chromosomes().append(child2)

    new_pop.get_chromosomes().sort(key=lambda x: x.get_fitness(), reverse=False)   
    return new_pop

In [22]:
generation_number = 0
MAX_FITNESS = 10
population = Population(POP_SIZE)
population.print_population(generation_number)

nb = 0
while population.get_chromosomes()[0].get_fitness() > MAX_FITNESS and nb < 3 :
  nb = nb + 1
  generation_number += 1
  population = evolve(population)
  population.print_population(generation_number)


-----------------------Generation Summary---------------------------
--------------------------------------------------------------------
Generation # 0 | Fittest chromosome fitness: 73.69
--------------------------------------------------------------------
Chromosome # 0  : [(6, 9), (1, 7), (0, 4), (0, 3), (6, 3), (5, 8), (7, 7), (9, 8), (0, 6), (7, 2), (5, 0), (3, 4), (5, 1), (8, 2), (0, 2)] | Fitness 73.69
Chromosome # 1  : [(0, 2), (5, 8), (5, 1), (0, 3), (3, 4), (7, 7), (6, 3), (6, 9), (0, 4), (1, 7), (0, 6), (9, 8), (8, 2), (7, 2), (5, 0)] | Fitness 75.38
Chromosome # 2  : [(3, 4), (5, 0), (1, 7), (5, 8), (6, 9), (0, 4), (8, 2), (7, 2), (5, 1), (0, 6), (9, 8), (6, 3), (7, 7), (0, 3), (0, 2)] | Fitness 76.28
Chromosome # 3  : [(5, 0), (7, 2), (0, 4), (6, 3), (7, 7), (5, 8), (1, 7), (9, 8), (6, 9), (3, 4), (0, 6), (5, 1), (0, 2), (8, 2), (0, 3)] | Fitness 81.4
Chromosome # 4  : [(7, 7), (5, 1), (5, 8), (6, 3), (3, 4), (1, 7), (5, 0), (6, 9), (0, 6), (0, 2), (0, 4), (0, 3), (8, 2),